In [ ]:
import os
import re

import numpy as np
import pandas as pd
import rioxarray
import xarray as xr

In [ ]:
# Base directory
base_dir = "/home/jovyan/grid4earth_S2L1B/Sentinel-2/MSI/MSI_L1B_GR/2025/07/24"

# Band suffixes of interest
band_ids = [
    "B01",
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B09",
    "B10",
    "B11",
    "B12",
    "B8A",
]

# List SAFE folders
safe_dirs = sorted(
    [
        d
        for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d)) and d.endswith(".11")
    ]
)

# Container for time-stamped datasets
ds_list = []

for safe_name in safe_dirs:
    safe_path = os.path.join(base_dir, safe_name)
    img_data_dir = os.path.join(safe_path, "IMG_DATA")

    print(f"\n📦 Processing: {safe_name}")

    # Extract sensing time from the filename, between "_S" and "_D"
    match = re.search(r"_S(\d{8}T\d{6})_D", safe_name)
    if not match:
        print(f"❌ No timestamp found in {safe_name}")
        continue

    timestamp_str = match.group(1)
    timestamp = pd.to_datetime(timestamp_str, format="%Y%m%dT%H%M%S")

    data_vars = {}
    ref_da = None

    for band in band_ids:
        band_files = [f for f in os.listdir(img_data_dir) if f.endswith(f"{band}.jp2")]
        if not band_files:
            print(f"⚠️  Missing {band} in {safe_name}")
            continue

        path = os.path.join(img_data_dir, band_files[0])
        da = rioxarray.open_rasterio(path, masked=True).squeeze()

        # Use the first band as the shape reference
        if ref_da is None:
            ref_da = da
        # No CRS or reprojection handled here

        data_vars[band] = da

    if not data_vars:
        print("🚫 No bands loaded, skipping.")
        continue

    # Create Dataset and add time as a coordinate
    ds = xr.Dataset(data_vars)
    ds = ds.expand_dims(time=[timestamp])
    ds_list.append(ds)

# Combine all time-stamped datasets
if ds_list:
    ds_all = xr.concat(ds_list, dim="time")
    print("\n✅ Combined dataset with time dimension:")
    print(ds_all)
    # Optionally save:
    # ds_all.to_zarr("Sentinel2_L1B_timeseries.zarr", mode="w")
else:
    print("\n🚫 No datasets to combine.")

In [ ]:
ds_all

In [ ]:
ds_all.B02.isel(time=3).plot()

In [ ]:
ds_all.to_zarr("s2l1b_tmp.zarr")

In [ ]:
xr.open_zarr("https://data-taos.ifremer.fr/GRID4EARTH/s2l1b_tmp.zarr")

In [ ]:
xr.open_zarr("s2l1b_tmp.zarr")

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
xml_path = "S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112807_D06_N05.11.SAFE/S2A_OPER_MTD_L1B_GR_2APS_20250724T145646_S20250724T112807_D06.xml"

tree = ET.parse(xml_path)
root = tree.getroot()

# No namespace: just find the tag!
el = root.find(".//EXT_POS_LIST")
if el is not None:
    pos_list = el.text.strip().split()
else:
    raise ValueError("Could not find EXT_POS_LIST in the XML.")

coords = np.array(list(map(float, pos_list))).reshape(-1, 3)  # shape: (5, 3)
latlons = coords[:4, :2]

# 10m band image shape
nrows, ncols = 2304, 2552

# Assign corners
UL = latlons[0]  # Upper Left
LL = latlons[1]  # Lower Left
LR = latlons[2]  # Lower Right
UR = latlons[3]  # Upper Right

# Bilinear interpolation
rows = np.linspace(0, 1, nrows)
cols = np.linspace(0, 1, ncols)
grid_rows, grid_cols = np.meshgrid(rows, cols, indexing="ij")

lat_grid = (
    (1 - grid_rows) * (1 - grid_cols) * UL[0]
    + grid_rows * (1 - grid_cols) * LL[0]
    + grid_rows * grid_cols * LR[0]
    + (1 - grid_rows) * grid_cols * UR[0]
)
lon_grid = (
    (1 - grid_rows) * (1 - grid_cols) * UL[1]
    + grid_rows * (1 - grid_cols) * LL[1]
    + grid_rows * grid_cols * LR[1]
    + (1 - grid_rows) * grid_cols * UR[1]
)

print("lat_grid shape:", lat_grid.shape)
print("lon_grid shape:", lon_grid.shape)

In [ ]:
lat_grid

In [ ]:
# Example usage:
# band_path = "S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112807_D06_N05.11.SAFE/IMG_DATA/S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112807_D06_B04.jp2"
# xml_path  = "S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112807_D06_N05.11.SAFE/S2A_OPER_MTD_L1B_GR_2APS_20250724T145646_S20250724T112807_D06.xml"
# ds = load_band_with_latlon(band_path, xml_path)
# print(ds)import numpy as np


def load_band_with_latlon(band_path, xml_path):
    """
    Returns an xarray.Dataset with:
      - band data (named after the band, e.g. 'B04'),
      - latitude grid,
      - longitude grid,
    for the given Sentinel-2 L1B band and its metadata XML.
    """
    # Load the band
    da = rioxarray.open_rasterio(band_path, masked=True)
    # Remove band dimension if present
    if "band" in da.dims and da.sizes["band"] == 1:
        da = da.squeeze("band")
    nrows, ncols = da.shape[-2], da.shape[-1]
    y_dim, x_dim = da.dims[-2], da.dims[-1]

    # Parse XML for the EXT_POS_LIST tag
    tree = ET.parse(xml_path)
    root = tree.getroot()
    el = root.find(".//EXT_POS_LIST")
    if el is None:
        raise ValueError(f"Could not find EXT_POS_LIST in {xml_path}")

    pos_list = el.text.strip().split()
    coords = np.array(list(map(float, pos_list))).reshape(-1, 3)
    latlons = coords[:4, :2]
    print(latlons)
    UL, LL, LR, UR = latlons

    # Interpolation
    rows = np.linspace(0, 1, nrows)
    cols = np.linspace(0, 1, ncols)
    grid_rows, grid_cols = np.meshgrid(rows, cols, indexing="ij")
    lat_grid = (
        (1 - grid_rows) * (1 - grid_cols) * UL[0]
        + grid_rows * (1 - grid_cols) * LL[0]
        + grid_rows * grid_cols * LR[0]
        + (1 - grid_rows) * grid_cols * UR[0]
    )
    lon_grid = (
        (1 - grid_rows) * (1 - grid_cols) * UL[1]
        + grid_rows * (1 - grid_cols) * LL[1]
        + grid_rows * grid_cols * LR[1]
        + (1 - grid_rows) * grid_cols * UR[1]
    )

    band_name = os.path.basename(band_path).split("_")[-1].split(".")[0]

    # Build xarray.Dataset (lat/lon must have dims y_dim, x_dim)
    ds = xr.Dataset(
        data_vars={
            band_name: (da.dims, da.values),
            "latitude": ((y_dim, x_dim), lat_grid),
            "longitude": ((y_dim, x_dim), lon_grid),
        },
        coords={y_dim: da[y_dim], x_dim: da[x_dim]},
    )
    return ds


detector = "D06"
# Example usage:
band_path = "Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112804_D06_N05.11/IMG_DATA/S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112804_D06_B02.jp2"
xml_path = "Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112804_D06_N05.11/S2A_OPER_MTD_L1B_GR_2APS_20250724T145646_S20250724T112804_D06.xml"

# ds = l
ds = load_band_with_latlon(band_path, xml_path)
ds
# print(ds)

In [ ]:
# 2804
# ds.chunk(512).to_zarr('S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112804_D06_N05.11_B02.zarr',mode='w')

In [ ]:
xr.open_zarr(
    "S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112804_D06_N05.11_B02.zarr"
)

In [ ]:
ds = xr.open_zarr(
    "https://data-taos.ifremer.fr/GRID4EARTH/S2A_OPER_MSI_L1B_GR_2APS_20250724T145646_S20250724T112804_D06_N05.11_B02.zarr"
)
# Pick your variables
b = ds["B02"]  # or any band
lat = ds["latitude"]
lon = ds["longitude"]

In [ ]:
# Your coordinates
lat0, lon0 = 48.4375, -4.7708

lat_grid = ds["latitude"].values
lon_grid = ds["longitude"].values

# Compute distance to Lanildut for every pixel
dist = np.sqrt((lat_grid - lat0) ** 2 + (lon_grid - lon0) ** 2)
iy, ix = np.unravel_index(np.argmin(dist), dist.shape)
print(f"Lanildut closest pixel: y={iy}, x={ix}")

In [ ]:
# Define window size
win = 256  # half size, so 512=256*2

# Ensure indices stay within image bounds
y1 = max(iy - win, 0)
y2 = min(iy + win, lat_grid.shape[0])
x1 = max(ix - win, 0)
x2 = min(ix + win, lat_grid.shape[1])

# Slice your data
b = ds["B02"].isel(y=slice(y1, y2), x=slice(x1, x2))
lat_win = ds["latitude"].isel(y=slice(y1, y2), x=slice(x1, x2))
lon_win = ds["longitude"].isel(y=slice(y1, y2), x=slice(x1, x2))

In [ ]:
import matplotlib.pyplot as plt

# Use the previous code to extract b, lat_win, lon_win
# plt.imsave('s2_window.png', b.values, cmap='gray')

# Flatten to exclude NaNs
vals = b.values.flatten()
vals = vals[np.isfinite(vals)]

vmin, vmax = np.percentile(vals, [10, 90])  # 2nd to 98th percentile stretch

img_norm = np.clip((b.values - vmin) / (vmax - vmin), 0, 1)

plt.imsave("s2_window.png", img_norm, cmap="gray")

In [ ]:
import folium

# Find the corners for the image overlay
lat_min, lat_max = float(lat_win.min()), float(lat_win.max())
lon_min, lon_max = float(lon_win.min()), float(lon_win.max())

# Map centered at Lanildut
m = folium.Map(location=[lat0, lon0], zoom_start=13, tiles="OpenStreetMap")

# Overlay the image
image_bounds = [[lat_min, lon_min], [lat_max, lon_max]]
folium.raster_layers.ImageOverlay(
    name="Sentinel-2 window",
    image="s2_window.png",
    bounds=image_bounds,
    opacity=0.6,
    interactive=True,
    cross_origin=False,
    zindex=1,
).add_to(m)

# Add a marker for Lanildut
folium.Marker([lat0, lon0], popup="Lanildut", icon=folium.Icon(color="red")).add_to(m)

m.save("lanildut_sentinel2_map.html")
m  # In Jupyter, this will display the interactive map!